In [1]:
import numpy as np
import cv2
import os
from matplotlib import pyplot as plt

In [2]:
image_paths = []
os.curdir
for file_name in os.listdir('./images/left'):
    image_paths.append('./images/left/'+ file_name)
print(image_paths)


['./images/left/left09.jpg', './images/left/left08.jpg', './images/left/left01.jpg', './images/left/left03.jpg', './images/left/left02.jpg', './images/left/left12.jpg', './images/left/left06.jpg', './images/left/left07.jpg', './images/left/left13.jpg', './images/left/left05.jpg', './images/left/left11.jpg', './images/left/left10.jpg', './images/left/left04.jpg']


In [3]:
# Arrays to store object points and image points from all the images.
total_object_points = []
total_image_points = []

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
object_points = np.zeros((6 * 9, 3), np.float32)
object_points[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)
success = 0
for image_path in image_paths:
    image = cv2.imread(image_path)
    
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    found, image_points = cv2.findChessboardCorners(
        image = gray_image,
        patternSize=(7, 6), 
        corners=None,
    )

    # If found, add object points, image points (after refining them)
    if found:
        
        image_points = cv2.cornerSubPix(
            image=gray_image,
            corners=image_points,
            winSize=(11, 11),
            zeroZone=(-1, -1),
            criteria=(
                cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER,
                30,
                0.01
            )
        )
        
        success = success + 1
        total_object_points.append(object_points)
        total_image_points.append(image_points)
    else:
        print(image_path)
        
    # Draw and display the corners        
    new_image = image.copy()
    cv2.drawChessboardCorners(
        image=new_image, 
        patternSize=(7, 6), 
        corners=image_points, 
        patternWasFound=found
    )    
        
    plt.subplot(1,2,1)
    plt.imshow(image)
    plt.title(image_path)
    plt.subplot(1,2,2)
    plt.imshow(new_image)
    plt.title(image_path)
    plt.show()        
    

print(success)

./images/left/left09.jpg


./images/left/left10.jpg


11


In [4]:
ret, camera_matrix, distortion_coefficients, rotation_vectors, translation_vectors = cv2.calibrateCamera(
    objectPoints=total_object_points,
    imagePoints=total_image_points,
    imageSize=(640,480),
    cameraMatrix=None,
    distCoeffs=None
)
ret

0.15540724005687173

In [75]:
new_camera_matrix, roi=cv2.getOptimalNewCameraMatrix(
    camera_matrix, 
    distortion_coefficients, 
    (w,h), 
    1,
    (w,h)
)
print(camera_matrix)
print(new_camera_matrix)

for image_path in image_paths:
    
    image = cv2.imread(image_path)
    
    h,  w = image.shape[:2]
    

    undistorted_image = cv2.undistort(
        src=image, 
        cameraMatrix=camera_matrix,
        distCoeffs=distortion_coefficients, 
        dst=None,
        newCameraMatrix=new_camera_matrix
    )
    
    x, y, w, h = roi
    undistorted_image = undistorted_image[y:y+h, x:x+w]
    
    plt.subplot(1,2,1)
    plt.imshow(image)
    plt.title(image_path)
    plt.subplot(1,2,2)
    plt.imshow(undistorted_image)
    plt.title(image_path)
    plt.show()

[[534.07088364   0.         341.53407553]
 [  0.         534.11914595 232.9456526 ]
 [  0.           0.           1.        ]]
[[462.29959106   0.         345.82843277]
 [  0.         458.94192505 234.72740344]
 [  0.           0.           1.        ]]


In [8]:
mean_error = 0
for i in range(len(total_object_points)):
    image_points, _ = cv2.projectPoints(
        total_object_points[i],
        rotation_vectors[i],
        translation_vectors[i],
        camera_matrix, 
        distortion_coefficients
    )
    error = cv2.norm(total_image_points[i], image_points, cv2.NORM_L2)/len(image_points)
    mean_error += error
print( 'total error: {}'.format(mean_error) )
print( 'mean error: {}'.format(mean_error/len(total_object_points)) )

NameError: name 'new_camera_matrix' is not defined

In [108]:
for i in translation_vectors:
    print(i)

[[-3.27916485]
 [ 2.15811689]
 [11.72833343]]
[[ 3.79739146]
 [ 0.89895018]
 [14.8593055 ]]
[[3.03685642]
 [2.74656588]
 [9.90005138]]
[[-2.12419184]
 [ 1.84550818]
 [12.81229187]]
[[-1.22339632]
 [ 3.52139948]
 [15.67311789]]
[[ 1.60364763]
 [ 3.81296375]
 [16.12054474]]
[[-5.94286695]
 [ 2.39629427]
 [16.77859026]]
[[ 1.81888151]
 [-4.2642919 ]
 [12.45728517]]
[[-1.13991523]
 [ 2.67424464]
 [ 9.56289277]]
[[-2.89985494]
 [ 2.62066993]
 [10.57699258]]
[[ 2.82321756]
 [ 2.22374307]
 [10.95762951]]
